In [1]:
!pip install geopy

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
import urllib.request
import json

## 데이터 불러오기

In [3]:
df = pd.read_excel('.\\ot_data.xlsx')
df

,옷체통(의류수거함) 소재지 현황,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,연번,동,지번 주소,도로명 주소,관리번호,비고
2,1,서초1동,1610-4,효령로67길 71-9,A-19,NaN
3,2,서초1동,1622-9,사임당로8길 29,A-14,NaN
4,3,서초1동,1648-1,사임당로19길 22,A-24,NaN
...,...,...,...,...,...,...
247,246,내곡동,106-7,염곡말길 19,D-67,NaN
248,247,내곡동,39-1,염곡안 3길 21,D-47,NaN
249,248,내곡동,145-2,염곡안길 56,D-70,NaN
250,249,내곡동,142,염곡안3길 11,D-69,NaN


## 전처리

In [4]:
df2 = df.rename(columns=df.iloc[1])
df2

,연번,동,지번 주소,도로명 주소,관리번호,비고
0,NaN,NaN,NaN,NaN,NaN,NaN
1,연번,동,지번 주소,도로명 주소,관리번호,비고
2,1,서초1동,1610-4,효령로67길 71-9,A-19,NaN
3,2,서초1동,1622-9,사임당로8길 29,A-14,NaN
4,3,서초1동,1648-1,사임당로19길 22,A-24,NaN
...,...,...,...,...,...,...
247,246,내곡동,106-7,염곡말길 19,D-67,NaN
248,247,내곡동,39-1,염곡안 3길 21,D-47,NaN
249,248,내곡동,145-2,염곡안길 56,D-70,NaN
250,249,내곡동,142,염곡안3길 11,D-69,NaN


In [5]:
# 불필요한 행 삭제
df2 = df2.drop(index=[0,1], axis=0)
df2

,연번,동,지번 주소,도로명 주소,관리번호,비고
2,1,서초1동,1610-4,효령로67길 71-9,A-19,NaN
3,2,서초1동,1622-9,사임당로8길 29,A-14,NaN
4,3,서초1동,1648-1,사임당로19길 22,A-24,NaN
5,4,서초1동,1438-15,효령로66길 11,A-01,NaN
6,5,서초1동,1610-20,효령로67길 49-2,A-20,NaN
...,...,...,...,...,...,...
247,246,내곡동,106-7,염곡말길 19,D-67,NaN
248,247,내곡동,39-1,염곡안 3길 21,D-47,NaN
249,248,내곡동,145-2,염곡안길 56,D-70,NaN
250,249,내곡동,142,염곡안3길 11,D-69,NaN


In [6]:
# 열삭제
df2 = df2.drop(['비고'], axis=1)
df2

,연번,동,지번 주소,도로명 주소,관리번호
2,1,서초1동,1610-4,효령로67길 71-9,A-19
3,2,서초1동,1622-9,사임당로8길 29,A-14
4,3,서초1동,1648-1,사임당로19길 22,A-24
5,4,서초1동,1438-15,효령로66길 11,A-01
6,5,서초1동,1610-20,효령로67길 49-2,A-20
...,...,...,...,...,...
247,246,내곡동,106-7,염곡말길 19,D-67
248,247,내곡동,39-1,염곡안 3길 21,D-47
249,248,내곡동,145-2,염곡안길 56,D-70
250,249,내곡동,142,염곡안3길 11,D-69


In [7]:
# 인덱스 재정렬
df2 = df2.reset_index(drop=True)
df2

,연번,동,지번 주소,도로명 주소,관리번호
0,1,서초1동,1610-4,효령로67길 71-9,A-19
1,2,서초1동,1622-9,사임당로8길 29,A-14
2,3,서초1동,1648-1,사임당로19길 22,A-24
3,4,서초1동,1438-15,효령로66길 11,A-01
4,5,서초1동,1610-20,효령로67길 49-2,A-20
...,...,...,...,...,...
245,246,내곡동,106-7,염곡말길 19,D-67
246,247,내곡동,39-1,염곡안 3길 21,D-47
247,248,내곡동,145-2,염곡안길 56,D-70
248,249,내곡동,142,염곡안3길 11,D-69


In [8]:
# 데이터타입 변경
df2 = df2.astype({'지번 주소':'str'})
df2

,연번,동,지번 주소,도로명 주소,관리번호
0,1,서초1동,1610-4,효령로67길 71-9,A-19
1,2,서초1동,1622-9,사임당로8길 29,A-14
2,3,서초1동,1648-1,사임당로19길 22,A-24
3,4,서초1동,1438-15,효령로66길 11,A-01
4,5,서초1동,1610-20,효령로67길 49-2,A-20
...,...,...,...,...,...
245,246,내곡동,106-7,염곡말길 19,D-67
246,247,내곡동,39-1,염곡안 3길 21,D-47
247,248,내곡동,145-2,염곡안길 56,D-70
248,249,내곡동,142,염곡안3길 11,D-69


In [9]:
df2['address'] = df2['동']+" "+df2['지번 주소']
df2

,연번,동,지번 주소,도로명 주소,관리번호,address
0,1,서초1동,1610-4,효령로67길 71-9,A-19,서초1동 1610-4
1,2,서초1동,1622-9,사임당로8길 29,A-14,서초1동 1622-9
2,3,서초1동,1648-1,사임당로19길 22,A-24,서초1동 1648-1
3,4,서초1동,1438-15,효령로66길 11,A-01,서초1동 1438-15
4,5,서초1동,1610-20,효령로67길 49-2,A-20,서초1동 1610-20
...,...,...,...,...,...,...
245,246,내곡동,106-7,염곡말길 19,D-67,내곡동 106-7
246,247,내곡동,39-1,염곡안 3길 21,D-47,내곡동 39-1
247,248,내곡동,145-2,염곡안길 56,D-70,내곡동 145-2
248,249,내곡동,142,염곡안3길 11,D-69,내곡동 142


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   연번       250 non-null    object
 1   동        250 non-null    object
 2   지번 주소    250 non-null    object
 3   도로명 주소   250 non-null    object
 4   관리번호     250 non-null    object
 5   address  250 non-null    object
dtypes: object(6)
memory usage: 11.8+ KB


## Geocoding

In [11]:
# 위경도 함수 정의
def geocoding(address):
    geocoder = Nominatim(user_agent='South Korea', timeout=None)
    geo = geocoder.geocode(address)
    xy = {'lat': str(geo.latitude), 'Ing': str(geo.longitude)}
    return xy


In [12]:
address_xy = geocoding('사임당로8길 29')
address_xy

{'lat': '37.4875469', 'Ing': '127.0155707'}

In [13]:
df2.iloc[186]

연번                187
동                양재1동
지번 주소           115-5
도로명 주소     강남대로23길 72
관리번호             D-32
address    양재1동 115-5
Name: 186, dtype: object

## Naver API Json Parsing

In [14]:
address = urllib.parse.quote_plus('서초2동 1376-3')

In [15]:
url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'

In [16]:
req = urllib.request.Request(url)

In [17]:
req.add_header('X-NCP-APIGW-API-KEY-ID', '88g5r3zt05')#Client Key 문자열로 입력

In [18]:
req.add_header('X-NCP-APIGW-API-KEY', 'jaUefykPS17u6sa2e9pwAzLwlHjrZ0RDpid9FCom')#Client Secret 문자열로 입력

In [19]:
response = urllib.request.urlopen(req)

In [20]:
response_code = response.getcode()#사이트 상태 확인

In [21]:
if response_code == 200:#http 정상 응답시
    response_body = response.read()
    #JSON format 데이터 Parsing
    map_data = json.loads(response_body)
    print(map_data)

{'status': 'OK', 'meta': {'totalCount': 1, 'page': 1, 'count': 1}, 'addresses': [{'roadAddress': '서울특별시 서초구 남부순환로 2584 서초구청, 서초구의회', 'jibunAddress': '서울특별시 서초구 서초2동 1376-3 서초구청, 서초구의회', 'englishAddress': '2584, Nambusunhwan-ro, Seocho-gu, Seoul, Republic of Korea', 'addressElements': [{'types': ['SIDO'], 'longName': '서울특별시', 'shortName': '서울특별시', 'code': ''}, {'types': ['SIGUGUN'], 'longName': '서초구', 'shortName': '서초구', 'code': ''}, {'types': ['DONGMYUN'], 'longName': '서초2동', 'shortName': '서초2동', 'code': ''}, {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['ROAD_NAME'], 'longName': '남부순환로', 'shortName': '남부순환로', 'code': ''}, {'types': ['BUILDING_NUMBER'], 'longName': '2584', 'shortName': '2584', 'code': ''}, {'types': ['BUILDING_NAME'], 'longName': '서초구청, 서초구의회', 'shortName': '서초구청, 서초구의회', 'code': ''}, {'types': ['LAND_NUMBER'], 'longName': '1376-3', 'shortName': '1376-3', 'code': ''}, {'types': ['POSTAL_CODE'], 'longName': '06750', 'shortName': '06750', 'co

In [22]:
print(len(map_data['addresses']))

1


In [23]:
lat = map_data['addresses'][0]['y']
lat

'37.4837460'

In [24]:
Ing = map_data['addresses'][0]['x']
Ing

'127.0325901'

In [25]:
df2.iloc[169]

연번                 170
동                 방배4동
지번 주소           840-28
도로명 주소     서초대로33길 63 
관리번호              C-49
address    방배4동 840-28
Name: 169, dtype: object

## 엑셀 데이터 위경도 변환

In [26]:
for i, row in df2.iterrows():
    road_ad = row['address']
    print(road_ad)

서초1동 1610-4
서초1동 1622-9
서초1동 1648-1
서초1동 1438-15
서초1동 1610-20
서초1동 1617-26
서초1동 1617-41
서초1동 1617-1
서초1동 1430-1
서초1동 1628-30
서초1동 1608-8
서초1동 1636-1
서초1동 1622-13
서초1동 1666-2
서초1동 1643-18
서초1동 1643-31
서초1동 1666-17
서초1동 1610-1
서초1동 1635-10
서초1동 1668-1
서초1동 1668-12
서초1동 1618-3
서초1동 1679-6
서초2동 1350
서초2동 1349
서초2동 1324-1
서초2동 1349-21
서초2동 1360-24
서초2동 1357-48
서초3동  1518-1 
서초3동 1451-66
서초3동 1451-76
서초3동 1466-4
서초3동 1467-70
서초3동 1469-6
서초3동 1487-63
서초3동 1467-75
서초3동 1474-14
서초3동 1487-139
서초3동 1487-19
서초3동 1475-3
서초3동 1503-14
서초3동 1520-1
서초3동 1507-60
서초3동 1508-44
서초3동 1508-9
서초3동 1512-9
서초3동 1519-8
서초3동 1522-8
서초3동 1523-8
서초3동 1529
서초3동 1532-10
서초3동 1546-5
서초3동 1547-15
서초3동 1561-1
서초3동 1565-18
서초3동 1562-9
서초3동 1567-1
잠원동 32-4
잠원동 10-34
잠원동 15-1
잠원동 23-18
잠원동 7-9
잠원동 44-1
잠원동 45-10
반포1동 709-5
반포1동 709-5
반포1동 728-47
반포1동 730
반포1동 728-26
반포4동 100-8
반포4동 101-7
반포4동 612-110
반포4동 74-7
반포4동 76-7
반포4동 81-10
반포4동 83-4
반포4동 84-13
반포4동 107-57
반포4동 591-1
반포4동 78-13
방배본동 12
방배본동 1-119
방배본동 12-35
방배본동 12-

In [39]:
address_x = []
address_y = []
for i, row in df2.iterrows():#데이터프레임 반복문
    jibun_ad = row['address']#지번주소
    doro_ad = row['도로명 주소']#도로명주소
#     print(jibun_ad)
#     print(i)
    address1 = urllib.parse.quote_plus(jibun_ad)
    address2 = urllib.parse.quote_plus(doro_ad)
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address1}'
    req = urllib.request.Request(url)
    req.add_header('X-NCP-APIGW-API-KEY-ID', '88g5r3zt05')#Client Key 문자열로 입력
    req.add_header('X-NCP-APIGW-API-KEY', 'jaUefykPS17u6sa2e9pwAzLwlHjrZ0RDpid9FCom')#Client Secret 문자열로 입력
    response = urllib.request.urlopen(req)
    response_code = response.getcode()#사이트 상태 확인
    if response_code == 200:#http 정상 응답시
        response_body = response.read()
        #JSON format 데이터 Parsing
        map_data = json.loads(response_body)
#         print(map_data)
#         print(i)
        if len(map_data['addresses']) != 0:#호출한 주소 데이터 갯수가 0이 아니면
            lat = map_data['addresses'][0]['y']
            lng = map_data['addresses'][0]['x']
            address_x.append(lng)#list 추가
            address_y.append(lat)
        else:
            url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address2}'
            req = urllib.request.Request(url)
            req.add_header('X-NCP-APIGW-API-KEY-ID', '88g5r3zt05')#Client Key 문자열로 입력
            req.add_header('X-NCP-APIGW-API-KEY', 'jaUefykPS17u6sa2e9pwAzLwlHjrZ0RDpid9FCom')#Client Secret 문자열로 입력
            response = urllib.request.urlopen(req)
            response_code = response.getcode()#사이트 상태 확인
            if response_code == 200:#http 정상 응답시
               response_body = response.read()
               map_data2 = json.loads(response_body)
               lat = map_data2['addresses'][0]['y']
               lng = map_data2['addresses'][0]['x']
               address_x.append(lng)#list 추가
               address_y.append(lat)

In [28]:
print(len(address_x))

250


In [29]:
print(len(address_y))

250


In [30]:
add_xy_df = pd.DataFrame(zip(address_y, address_x), columns = ['lat', 'lng'])
add_xy_df

,lat,lng
0,37.4899141,127.0223614
1,37.4873589,127.0159569
2,37.4909598,127.0194595
3,37.4862195,127.0227621
4,37.4894139,127.0227200
...,...,...
245,37.6386643,126.8095682
246,37.4514169,127.0691117
247,37.6424337,126.7978142
248,35.8875615,128.7603215


In [31]:
add_xy_df['lat_lng'] = add_xy_df['lat']+","+add_xy_df['lng']
add_xy_df

,lat,lng,lat_lng
0,37.4899141,127.0223614,"37.4899141,127.0223614"
1,37.4873589,127.0159569,"37.4873589,127.0159569"
2,37.4909598,127.0194595,"37.4909598,127.0194595"
3,37.4862195,127.0227621,"37.4862195,127.0227621"
4,37.4894139,127.0227200,"37.4894139,127.0227200"
...,...,...,...
245,37.6386643,126.8095682,"37.6386643,126.8095682"
246,37.4514169,127.0691117,"37.4514169,127.0691117"
247,37.6424337,126.7978142,"37.6424337,126.7978142"
248,35.8875615,128.7603215,"35.8875615,128.7603215"


In [32]:
add_xy_df.iloc[169]

lat                    37.4912152
lng                   126.9953132
lat_lng    37.4912152,126.9953132
Name: 169, dtype: object

## 지도 시각화

In [33]:
import folium

In [40]:
# 위도
latitude = 37.4837460
# 경도
longitude = 127.0325901

In [35]:
# 서초구 지도 생성
m = folium.Map(location=[latitude, longitude],
               zoom_start=13, 
               width=750, 
               height=500
              )
m

In [43]:
for i, row in add_xy_df.iterrows():
    latitude = row['lat']
    longitude = row['lng']
    folium.Marker([latitude, longitude],
                  icon=folium.Icon('lightred', icon='star'),
                 ).add_to(m)
m